# ADB命令详解
　　参考：[ADB常用命令](https://juejin.cn/post/7027766928211083295)  
　　　　　[ADB LogCat抓取日志全解析](https://blog.csdn.net/qq_24373725/article/details/79142869)   
　　　　　[adb logcat说明](https://blog.csdn.net/jasonactions/article/details/108683379)  
　　　　　[adb链接真机unauthorized](https://www.cnblogs.com/Eliphaz/p/13536231.html)    
　　　　　[]()   
　　　　　[]()   
　　　　　[]()   
[Android adb安装时强制应用App以32位或者64位运行](https://blog.csdn.net/tkwxty/article/details/107608252)  
 　　强制应用以32位运行看看是否OK，如果OK，则APK没有适配64位的so库。
　　应用以32位在64位终端环境下运行，先上命令格式如下：
>adb install --abi armeabi-v7a \<path to apk\>

应用以64位在64位终端环境下运行,先上命令格式如下：
>adb install --abi arm64-v8a \<path to apk\>

其他的Android平台，譬如X86或者其它的，可以使用如下命令：
>adb install --abi \<path to apk\>
  
 
    
## adb指定安装设备
　-d   指定当前唯一通过USB连接的Android设备为命令目标   
　-e   指定当前唯一运行的模拟器为命令目标   
　-s<serialno>   指定相应的serialno号的设备或模拟器为命令目标   
```shell
adb -s G2W02186660113443 install /Users/liing/Desktop/app/app.apk      # G2W02186660113443 指定的安装设备
adb shell pm list packages -s     # 查询系所有系统应用的包名
adb shell pm list packages -3     # 查询所有第三方应用包名
adb shell pm path +apk包名         # 获取apk的安装路径
adb shell pm clear （apk包名）     # 清除应用数据与缓存
adb bugreport >d:\bugreport.txt   # 查看bug报告
adb shell am start -n com.helloshan.demo/.MianActivity   # 启动某个activity
adb shell am force-stop （apk包名）  # 强制停止应用
db shell dumpsys activity         # adb查看当前页面的activity:输出中mResumedActivity对应的activity
```
## adb 获取包名
```shell
adb shell dumpsys window | findstr mCurrentFocus    # 输出包名/打开的activity
```
 
## adb重启服务
```shell
adb kill-server  
adb start-server  # 或者直接使用adb操作命令，也可以启动adb服务   
```
    
## adb logcat命令
　　用于输出设备日志,分析错误原因。  
　　adb logcat 命令，从以下四部分组成理解，分别是命令格式，常用命令，shell脚本输出指定app日志，android系统log缓冲区。
### 命令格式
　　adb logcat [选项] [过滤项], 其中 选项 和 过滤项 在 中括号[]中, 说明这是可选的;
　　选项解析:
1. "-s"选项 : 只显示指定标签的日志; ------>adb logcat -s SWVDEC 显示SWVDEC标签的日志
2.　"-v"选项 : 设置日志的输出格式;----->adb logcat -v threadtime 查看日志输出时间和线程信息
3.　"-c"选项 : 清空所有的日志缓存信息;---->adb logcat -c
4.　"-d"选项 : 将缓存的日志输出到屏幕上, 并且不会阻塞;------->adb logcat -d
5.　"-t"选项 : 输出最近的几行日志, 输出完退出, 不阻塞;------>adb logcat -t 5 输出日志缓冲区的最近5行
6.　"-g"选项 : 查看日志缓冲区信息; ------>adb logcat -g
7.　"-B"选项 : 以二进制形式输出日志; ----> adb logcat -B
8. "-f"选项 : 该选向后面跟着输入日志的文件, 使用adb logcat -f /sdcard/log.txt 命令, 注意这个log文件是输出到手机上，需要指定合适的路径。这个参数对对不能一直用电脑连着手机收集日志的场景非常有用，其实Android shell下也有一个相同参数的logcat命令。使用如下命令可以执行后断开PC和手机持续收集LOG。如：logcat -f /sdcard/log.txt &  后台收集。

### 常用命令
1. 过滤指定等级日志 比如: 显示Error以上级别的日志: adb logcat \*:E 常用的还有D (debug以上级别), I (Info以上级别)
2. log输出到计算机文件 adb logcat > /home/cherish/log.txt ( 相对路径.绝对路径都可以.)
3. 使用管道过滤日志，如：过滤指定标签"android"或者"system"标签的log,并输出到文件，adb logcat | grep -E “android|system” > ~/log.txt
4. 将日志保存到文件 , 但无法输出到屏幕 , 针对这个问题可以采用 tee 命令。adb logcat | grep -E “android|system” | tee ~/log.txt
5. 给指定标签文字加颜色。如：adb logcat | grep --color=auto -i android
过滤带有标签"android"的log,并给"android"字样加上颜色。
6. 输出带有某个包名的日志。如：adb logcat | grep “需要过滤的包名”
7. adb logcat -help 如果具体命令没记住 , 使用它查看简单的命令格式。

　　注:
+ 在 windows 上不能使用 grep 关键字,可以用 findstr 代替 grep。
logcat -f /sdcard/log.txt & 
+ 另外查看手机启动时候的内核日志命令: adb shell dmesg  



### shell脚本输出指定app日志
　　思路: 先通过包名找到该进程的pid, 使用pid进行过滤该应用日志. 代码如下:
```shell
#!/bin/bash
packageName=$1

systemVersion="$(adb shell getprop ro.build.version.release)"
echo "systemVersion=$systemVersion"
version=`echo $systemVersion | awk -F '.' '{print $1}'`

ver=`expr $version + 0`  
#android8.0及以上系统获取正在运行的进程pid需要加上-A:  adb shell ps -A
if [ $ver -ge 8 ]; then
   pid="$(adb shell ps -A | grep $packageName | awk '{print $2}')" 
else
   pid="$(adb shell ps | grep $packageName | awk '{print $2}')"
fi

echo "pid=$pid"

echo "-----------------------------"
adb logcat | grep --color=auto $pid
```
　　执行脚本命令 ./log.sh 包名    
　　比如 ./log.sh com.android.systemui

### android系统log缓冲区
　　　参考：[adb logcat 命令](https://blog.csdn.net/zhaohuih/article/details/106905219)   
　　加载日志缓冲区的日志: 使用 adb logcat -b 缓冲区类型 命令;   
　　Android中的日志缓冲区 :  
+ system缓冲区 - 与系统相关的日志信息
+ radio缓冲区 - 广播电话相关的日志信息
+ events缓冲区 - 事件相关的日志信息
+ main缓冲区 - 默认的缓冲区

　　其他非缓冲区参数：  
+ crash	看名字就知道啥意思
+ kernel	内核日志 可以看到 rc文件中服务启动情况
+ default	主要输出 main,system,crash 三个缓冲区中的日志
+ all	以上所有缓冲区的日志全部输出

#### 1、android log系统组成
#### 2、logcat缓冲区
android系统把log输出到不同的缓冲区中，目前定义了四个log缓冲区 .
+ radio：输出通信系统(无线/电话相关)的log
+ system：输出系统组件的log
+ events：输出事件相关模块的log
+ main：所有java层的log, 也是主缓冲区 (默认缓冲区)

　　缓冲区主要给系统组件使用，上层应用不需要关心，应用的log都输出到main缓冲区中
　　默认log输出（不指定缓冲区的情况下）是输出System和Main缓冲区的log
